In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy # data prcessing 
import tensorflow as tf 
from tensorflow import keras
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-dataset-sentiment-analysis/EcoPreprocessed.csv


In [2]:
df = pd.read_csv('/kaggle/input/preprocessed-dataset-sentiment-analysis/EcoPreprocessed.csv')
df[df.division == 'negative'].shape
df.shape

(4084, 4)

In [3]:
lemmatization = spacy.load("en_core_web_sm")
df.review = df['review'].apply((lambda row: " ".join([w.lemma_ for w in lemmatization(row)])))



In [4]:
df['division'].unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [5]:
tokenize = keras.preprocessing.text.Tokenizer(num_words=1000)
tokenize.fit_on_texts(df.review)
bag_to_words = tokenize.texts_to_matrix(df.review)
bag_to_words = pd.DataFrame(bag_to_words)

In [6]:
label = pd.get_dummies(data = df['division'],columns=['division'])

features = [bag_to_words, df['polarity']]
full_dataset = pd.concat(features, axis=1)
vocab_size = full_dataset.shape[1]
full_dataset.shape

(4084, 1001)

In [7]:
'''from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(full_dataset,df['division'] )
X_over.head() '''

"from imblearn.over_sampling import RandomOverSampler\noversample = RandomOverSampler(sampling_strategy='minority')\nX_over, y_over = oversample.fit_resample(full_dataset,df['division'] )\nX_over.head() "

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(full_dataset, label)

In [9]:
model = keras.Sequential()
model.add(keras.layers.Dense(vocab_size, input_shape= (vocab_size,), activation='relu'))
model.add(keras.layers.Dense(int(vocab_size/2), activation='relu'))
model.add(keras.layers.Dense(25, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

2022-07-20 12:03:42.394650: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss = "binary_crossentropy",
              metrics = METRICS)

In [11]:

model.fit(x_train,y_train, epochs=100);

2022-07-20 12:03:42.698991: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
96/96 [==============================] - 2s 8ms/step - loss: 0.2584 - accuracy: 0.8878 - precision: 0.8794 - recall: 0.7689
Epoch 2/100
96/96 [==============================] - 1s 8ms/step - loss: 0.0775 - accuracy: 0.9753 - precision: 0.9708 - recall: 0.9546
Epoch 3/100
96/96 [==============================] - 1s 8ms/step - loss: 0.0139 - accuracy: 0.9962 - precision: 0.9944 - recall: 0.9941
Epoch 4/100
96/96 [==============================] - 1s 8ms/step - loss: 0.0039 - accuracy: 0.9996 - precision: 0.9993 - recall: 0.9993
Epoch 5/100
96/96 [==============================] - 1s 7ms/step - loss: 0.0010 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 6/100
96/96 [==============================] - 1s 8ms/step - loss: 8.9746e-04 - accuracy: 0.9998 - precision: 0.9997 - recall: 0.9997
Epoch 7/100
96/96 [==============================] - 1s 8ms/step - loss: 0.0011 - accuracy: 0.9998 - precision: 0.9997 - recall: 0.9997
Epoch 8/100
96/96 [=========================

In [12]:
model.predict(x_test)

result =  model.evaluate(x_test, y_test)
#classes_x[480:589]
print(f'accuracy:{result[1]}\nprecision:{result[2]}\nrecall:{result[3]}')

32/32 [==============================] - 1s 4ms/step - loss: 0.3076 - accuracy: 0.9667 - precision: 0.9500 - recall: 0.9500
accuracy:0.9666993618011475
precision:0.9500489830970764
recall:0.9500489830970764
